In [4]:
import ee
import numpy as np
import folium
import json
from util import shp_to_ee_fmt,add_ee_layer,get_whole_region
import time 
from geetools import tools
import geopandas as gpd
import pandas as pd
from yearly_bar_plot_tree_cover import pixel_stats_dynamic_world,plot_multiple_bar_from_dict,pixel_stats_world_cover,pixel_stats_global_forest_watch
from constants import * 
import matplotlib.pyplot as plt


try:     
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

In [195]:
# TO EXPROT CSV OF YEAR FOR EACH REGION WITH WC


shape_file = '/Users/gclyne/Downloads/rfcx_sensor_placement/Graham_Amazon/Bacajai_JP1.shp'
file_name = shape_file.split('/')[-1].split('.')[0]
start_date = '2017-01-01'
end_date = '2021-01-01'
shp_file = gpd.read_file(shape_file, crs='EPSG:4326')
trees = []
data = json.loads(shp_file.to_json())

    #for each feature in image, get monthly avg and get band values
for index in range(len(data['features'])):
    print(f'processing image: {index}')
    # #get image of region
    region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,index))

    dataset = ee.ImageCollection("ESA/WorldCover/v100").first().clip(region)
    pixelCountStats = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=10)
    total = sum(pixelCountStats.getInfo()['Map'].values())

    output = pixelCountStats.getInfo()['Map']['10'] / total
    # for key,value in data.items():
    #     print(key,value)
    #     sum = 0
    #     for val in value['label'].values():
    #         print(val)
    #         sum = sum + val
    #     new_dict[key] = {'1':value['label']['1'] / sum}
    # # print(new_dict)
    # datas.append(new_dict)
    trees.append(output)
df_wc = pd.DataFrame({'id':range(len(data['features'])),'2020':trees})
df_wc
df_wc.to_csv('wc_2020_tembe.csv')

# pixelCountStats = dw.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=1)
# output = pixelCountStats.getInfo()
# print(output)
#

processing image: 0
processing image: 1


In [89]:
#make CSV for GFC 

shape_file = '/Users/gclyne/Downloads/rfcx_sensor_placement/Graham_Amazon/Bacajai_JP1.shp'
file_name = shape_file.split('/')[-1].split('.')[0]
start_date = '2017-01-01'
end_date = '2021-01-01'
shp_file = gpd.read_file(shape_file, crs='EPSG:4326')
trees = []
data = json.loads(shp_file.to_json())

    #for each feature in image, get monthly avg and get band values
for index in range(len(data['features'])):
    print(f'processing image: {index}')
    # #get image of region
    region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,index))

    dataset = ee.ImageCollection("UMD/hansen/global_forest_change_2021_v1_9").first().clip(region)
    pixelCountStats = dataset.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=10)
    total = sum(pixelCountStats.getInfo()['Map'].values())

    output = pixelCountStats.getInfo()['Map']['10'] / total
    # for key,value in data.items():
    #     print(key,value)
    #     sum = 0
    #     for val in value['label'].values():
    #         print(val)
    #         sum = sum + val
    #     new_dict[key] = {'1':value['label']['1'] / sum}
    # # print(new_dict)
    # datas.append(new_dict)
    trees.append(output)
df_wc = pd.DataFrame({'id':range(len(data['features'])),'2020':trees})
df_wc
df_wc.to_csv('wc_2020_tembe.csv')

,2017,2018,2019,2020
id,,,,
0,0.758592,0.758911,0.752994,0.765678
1,0.746441,0.748631,0.750771,0.740572
2,0.758592,0.758911,0.752994,0.765678
3,0.746441,0.748631,0.750771,0.740572
4,0.758592,0.758911,0.752994,0.765678
5,0.746441,0.748631,0.750771,0.740572
6,0.758592,0.758911,0.752994,0.765678
7,0.746441,0.748631,0.750771,0.740572
8,0.758592,0.758911,0.752994,0.765678


In [91]:
import plotly.express as px
fin_df = fin_df.reset_index()
px.bar(fin_df,x='id',y=['2017','2018','2019','2020'],barmode='group')

In [168]:
shape_file = '/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp', crs='EPSG:4326')
folium.Map.add_ee_layer = add_ee_layer    
file_name = shape_file.split('/')[-1].split('.')[0]  

my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,1))

years = ['2017','2018','2019','2020','2021']
for year_index in range(4):
    dw = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(years[year_index],years[year_index+1]).filterBounds(region).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    classification = ee.Image(dw.first()).clip(region) #here we use mode reduction, takes the most common pixel value over time period
    s2VisParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}    
    my_map.add_ee_layer(classification, s2VisParams, 'Classified Image')
    my_map.save(file_name + str(year_index) + '_s2_copernicus_first_single_dw.html')



In [174]:
shape_file = '/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp', crs='EPSG:4326')
folium.Map.add_ee_layer = add_ee_layer    
file_name = shape_file.split('/')[-1].split('.')[0]  

my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,0))


years = ['2017','2018','2019','2020','2021']
for year_index in range(4):
    dw = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterDate(years[year_index],years[year_index+1]).filterBounds(region)
    dwImage = ee.Image(dw.mode()).clip(region) #here we use mode reduction, takes the most common pixel value over time period
    classification = dwImage.select('label')
    dwVisParams = {'min': 0,'max': 8,'palette': ['#419BDF', '#397D49', '#88B053', '#7A87C6', '#E49635', '#DFC35A','#C4281B', '#A59B8F', '#B39FE1']}
    my_map.add_ee_layer(classification, dwVisParams, 'Classified Image')
    my_map.save(file_name + str(year_index) + '_mode_dw.html')

In [183]:
shape_file = '/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp'
shp_file = gpd.read_file('/Users/gclyne/Downloads/RFCx_GQ_Shapefiles/Hulu_Batang_Hari_sites_Buffer1k.shp', crs='EPSG:4326')
folium.Map.add_ee_layer = add_ee_layer    
file_name = shape_file.split('/')[-1].split('.')[0]

my_map = folium.Map(location=[region.getInfo()['coordinates'][0][0][1],region.getInfo()['coordinates'][0][0][0]], zoom_start=13, height=500)
region = ee.Geometry.Polygon(shp_to_ee_fmt(shp_file,20))


dw = ee.ImageCollection('ESA/WorldCover/v100').first().clip(region)
visualization = {'bands': ['Map']}
my_map.add_ee_layer(dw, visualization, 'Classified Image')
my_map.save(file_name + '_max_wc.html')

dw = ee.ImageCollection('ESA/WorldCover/v100').filterBounds(region).first().clip(region)
pixelCountStats = dw.reduceRegion(reducer=ee.Reducer.frequencyHistogram(),geometry=region,bestEffort=True,maxPixels=1e8,scale=1)
print(pixelCountStats.getInfo()['Map'])

{'10': 3149578.9686274487, '20': 271, '30': 12277, '40': 1045}
